In [1]:
%run "/Users/shubhi.singal@insight.com/LOSPredict" 

In [2]:
#encounter_df = pd.read_csv('encounters.csv').drop(columns=['Unnamed: 0'])
encounter_df.head()

,index,PATIENT_ID,PATIENT_ENCOUNTER_ID,ADMIT_TIME,ADMIT_ATTR,DISCHARGE_TIME,DISCHARGE_ATTR,PATIENT_AGE,LOS,MIDPOINT_TIME,ELECTIVE,EMERGENCY,NEWBORN,URGENT
0,301894,58526,100001,2117-09-11T11:46:00.000Z,EMERGENCY,2117-09-17T16:45:00.000Z,HOME,35.498630,6.207639,2117-09-14 14:15:30,0,1,0,0
1,47103,54610,100003,2150-04-17T15:34:00.000Z,EMERGENCY,2150-04-21T17:30:00.000Z,HOME,59.950685,4.080556,2150-04-19 16:32:00,0,1,0,0
2,135626,9895,100006,2108-04-06T15:49:00.000Z,EMERGENCY,2108-04-18T17:18:00.000Z,HOME,48.947945,12.061806,2108-04-12 16:33:30,0,1,0,0
3,165528,23018,100007,2145-03-31T05:33:00.000Z,EMERGENCY,2145-04-07T12:40:00.000Z,HOME,73.871233,7.296528,2145-04-03 21:06:30,0,1,0,0
4,308254,533,100009,2162-05-16T15:56:00.000Z,EMERGENCY,2162-05-21T13:37:00.000Z,HOME HEALTH CARE,60.835616,4.903472,2162-05-19 02:46:30,0,1,0,0


In [3]:
def to_datetime(pg_time_str):
    return datetime.strptime(pg_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")

In [4]:
last_seen = {}
readmitted = {}
encounter_df.sort_values(by='DISCHARGE_TIME', ascending=False, inplace=True)

In [5]:
for row in encounter_df.itertuples():
    d_t = row.DISCHARGE_TIME
    a_t = row.ADMIT_TIME
    pid = row.PATIENT_ID
    enc = row.PATIENT_ENCOUNTER_ID
    if pid not in last_seen:
        #never seen this patient before (last visit to hosp)
        last_seen[pid] = a_t
        readmitted[enc] = 2 #Bogus code: impossible to know readmission status
    else:
        #see how long in the future the readmission was
        td = to_datetime(last_seen[pid]) - to_datetime(d_t)
        if td.days < 30:
            #a 30-day readmission
            readmitted[enc] = 1
            last_seen[pid] = a_t
        else:
            readmitted[enc] = 0
            last_seen[pid] = a_t

In [6]:
encounters = encounter_rows_disch 
encounters.columns
#encounters.head()

Out[74]: Index(['Abnormal Findings', 'Blood/Immune Diseases', 'Circulatory',
 'Congenital Malformations', 'Digestive', 'Eye/Ear', 'Genitourinary',
 'Health Status/Services', 'Infectious/Parasitic Diseases',
 'Localized Injuries', 'Malignant Neoplasms',
 'Mental, Behavioral, Neurodevelopmental',
 'Musculoskeletal/Connective Tissue', 'Nervous System', 'Newborn',
 'Other External Morbidity', 'Other Injuries/Poisoning',
 'Other Neoplasms', 'Pregnancy', 'Respiritory', 'Skin',
 'Transport Accident', 'PATIENT_ENCOUNTER_ID', 'PATIENT_AGE', 'LOS',
 'ADMIT_ATTR_EMERGENCY', 'ADMIT_ATTR_ELECTIVE', 'ADMIT_ATTR_NEWBORN',
 'ADMIT_ATTR_URGENT', 'DISCHARGE_ATTR_DISC-TRAN CANCER/CHLDRN H',
 'DISCHARGE_ATTR_DISC-TRAN TO FEDERAL HC',
 'DISCHARGE_ATTR_DISCH-TRAN TO PSYCH HOSP', 'DISCHARGE_ATTR_HOME',
 'DISCHARGE_ATTR_HOME HEALTH CARE',
 'DISCHARGE_ATTR_HOME WITH HOME IV PROVIDR',
 'DISCHARGE_ATTR_HOSPICE-HOME',
 'DISCHARGE_ATTR_HOSPICE-MEDICAL FACILITY', 'DISCHARGE_ATTR_ICF',
 'DISCHARGE_ATTR_LEFT AGAINST MEDICAL ADVI',
 'DISCHARGE_ATTR_LONG TERM CARE HOSPITAL',
 'DISCHARGE_ATTR_OTHER FACILITY',
 'DISCHARGE_ATTR_REHAB/DISTINCT PART HOSP',
 'DISCHARGE_ATTR_SHORT TERM HOSPITAL', 'DISCHARGE_ATTR_SNF',
 'DISCHARGE_ATTR_SNF-MEDICAID ONLY CERTIF'],
 dtype='object')

In [7]:
encounters['30_DAY_READMIT'] = encounters.apply(lambda row: readmitted[row.PATIENT_ENCOUNTER_ID], axis=1)
encounter_df['30_DAY_READMIT'] = encounter_df.apply(lambda row: readmitted[row.PATIENT_ENCOUNTER_ID], axis=1)

In [8]:
def get_readmit_rate(row):
    pid = row.PATIENT_ID
    eid = row.PATIENT_ENCOUNTER_ID
    t = to_datetime(row.ADMIT_TIME)
    app_enc = kr_enc_df.loc[encounter_df.PATIENT_ID == pid] #applicable encounters
    
    app_enc = app_enc.loc[map(lambda other_time: to_datetime(other_time) < t, app_enc.ADMIT_TIME)]
    return np.nan_to_num(app_enc['30_DAY_READMIT'].mean())

In [9]:
kr_enc_df = encounter_df.loc[(encounter_df['30_DAY_READMIT'] < 2)] #known readmit
encounters = encounters.loc[(encounters['30_DAY_READMIT'] < 2)]

In [10]:
encounters['PATIENT_READMIT_RATE'] = kr_enc_df.apply(get_readmit_rate, axis=1)
encounters['PATIENT_READMIT_RATE'] = np.nan_to_num(encounters['PATIENT_READMIT_RATE'])

/local_disk0/tmp/1583759196823-0/PythonShell.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from __future__ import absolute_import
/local_disk0/tmp/1583759196823-0/PythonShell.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from __future__ import print_function

In [11]:
encounters.describe()

,Abnormal Findings,Blood/Immune Diseases,Circulatory,Congenital Malformations,Digestive,Eye/Ear,Genitourinary,Health Status/Services,Infectious/Parasitic Diseases,Localized Injuries,Malignant Neoplasms,"Mental, Behavioral, Neurodevelopmental",Musculoskeletal/Connective Tissue,Nervous System,Newborn,Other External Morbidity,Other Injuries/Poisoning,Other Neoplasms,Pregnancy,Respiritory,Skin,Transport Accident,PATIENT_ENCOUNTER_ID,PATIENT_AGE,LOS,ADMIT_ATTR_EMERGENCY,ADMIT_ATTR_ELECTIVE,ADMIT_ATTR_NEWBORN,ADMIT_ATTR_URGENT,DISCHARGE_ATTR_DISC-TRAN CANCER/CHLDRN H,DISCHARGE_ATTR_DISC-TRAN TO FEDERAL HC,DISCHARGE_ATTR_DISCH-TRAN TO PSYCH HOSP,DISCHARGE_ATTR_HOME,DISCHARGE_ATTR_HOME HEALTH CARE,DISCHARGE_ATTR_HOME WITH HOME IV PROVIDR,DISCHARGE_ATTR_HOSPICE-HOME,DISCHARGE_ATTR_HOSPICE-MEDICAL FACILITY,DISCHARGE_ATTR_ICF,DISCHARGE_ATTR_LEFT AGAINST MEDICAL ADVI,DISCHARGE_ATTR_LONG TERM CARE HOSPITAL,DISCHARGE_ATTR_OTHER FACILITY,DISCHARGE_ATTR_REHAB/DISTINCT PART HOSP,DISCHARGE_ATTR_SHORT TERM HOSPITAL,DISCHARGE_ATTR_SNF,DISCHARGE_ATTR_SNF-MEDICAID ONLY CERTIF,30_DAY_READMIT,PATIENT_READMIT_RATE
count,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.00000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.00000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.000000,12388.0,12388.000000,12388.000000
mean,0.563933,1.469729,2.848482,0.038344,0.899419,0.060462,0.734501,1.118179,0.463029,0.14062,0.198337,0.471263,0.328544,0.374717,0.056587,0.224169,0.229577,0.650387,0.003229,0.911285,0.176138,0.013158,150073.012270,72.956791,11.139061,0.824104,0.133032,0.018566,0.024298,0.017113,0.000161,0.005247,0.283662,0.272925,0.002583,0.001211,0.000081,0.00113,0.011624,0.066758,0.001292,0.149984,0.011866,0.174362,0.0,0.269374,0.020956
std,0.905941,1.521559,2.281381,0.222419,1.327731,0.276894,0.942514,1.394235,0.822902,0.72723,0.608044,0.877624,0.675657,0.720176,0.561783,0.530683,0.517482,0.849108,0.123153,1.156290,0.500719,0.113955,28897.490839,55.163028,12.386138,0.380747,0.339623,0.134993,0.153978,0.129699,0.012706,0.072249,0.450793,0.445481,0.050761,0.034778,0.008985,0.03360,0.107191,0.249613,0.035917,0.357070,0.108289,0.379436,0.0,0.443652,0.126263
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100001.000000,0.000000,0.023611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,125259.000000,51.793836,4.270660,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150282.000000,64.191781,7.361458,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,1.000000,2.000000,4.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000

In [12]:
encounters.columns[encounters.isna().any()].tolist()

Out[80]: []

In [13]:
x = encounters.drop(columns=['PATIENT_ENCOUNTER_ID', '30_DAY_READMIT']).to_numpy()
y = encounters.filter(['30_DAY_READMIT']).to_numpy()
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=11)

In [14]:
print(x.shape)
print(y.shape)

(12388, 45)
(12388, 1)

In [15]:
y = np.ravel(y,order='C')
#x = np.ravel(x,order='C')

In [16]:
print(x.shape)
print(y.shape)

(12388, 45)
(12388,)

In [17]:
y_train = np.ravel(y_train,order='C')
y_test = np.ravel(y_test,order='C')

In [18]:
xgb = XGBClassifier()
xgb.fit(x_train, y_train)
xgb_preds = xgb.predict_proba(x_test)

In [19]:
display(skplt.metrics.plot_roc_curve(y_test, xgb_preds))

In [20]:
y_test.shape
xgb_preds_1 = [row[1] for row in xgb_preds]
roc_auc_score(y_test, xgb_preds_1)

Out[88]: 0.6037115061303153

In [21]:
#Take a look at the xgb feature importances
xgb_fi = xgb.feature_importances_
cols = encounters.drop(columns=['PATIENT_ENCOUNTER_ID', '30_DAY_READMIT']).columns
sns.set(rc={'figure.figsize':(11.7,8.27)})
display(sns.barplot(x=xgb_fi, y=cols))

In [22]:
#the importance of emergency is interesting. What's the correlation between emergency admissions and 30-day readmits?
#perhaps easier: how often do emergency cases get readmitted?
base_readmit_rate = encounters['30_DAY_READMIT'].mean()
print(base_readmit_rate)
emergency_readmit_rate = encounters.loc[encounters.ADMIT_ATTR_EMERGENCY == 1]['30_DAY_READMIT'].mean()
print(emergency_readmit_rate)


0.2693735873425896
0.2657459104711529

In [23]:
#similarly interesting is the importance of genitourinary diagnosis
gu_readmit_rate = encounters.loc[encounters['Genitourinary'] > 0]['30_DAY_READMIT'].mean()
print(gu_readmit_rate)

0.2773562636992076

In [24]:
#are there any features that are highly important because they predict LOW readmission rates?
inf_readmit_rate = encounters.loc[encounters['Infectious/Parasitic Diseases'] > 0]['30_DAY_READMIT'].mean()
print(inf_readmit_rate)
resp_readmit_rate = encounters.loc[encounters['Respiritory'] > 0]['30_DAY_READMIT'].mean()
print(resp_readmit_rate)
ltc_readmit_rate = encounters.loc[encounters['DISCHARGE_ATTR_LONG TERM CARE HOSPITAL'] > 0]['30_DAY_READMIT'].mean()
print(ltc_readmit_rate)

0.28310263776442934
0.28083612573799266
0.4026602176541717

In [25]:
#might as well try some other classifiers
lgr = LogisticRegression()
lgr.fit(preprocessing.scale(x_train), y_train)
lgr_preds = lgr.predict_proba(preprocessing.scale(x_test))
lgr_preds_1 = [row[1] for row in lgr_preds]
roc_auc_score(y_test, lgr_preds_1)

Out[93]: 0.6420977364989543

In [26]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
knn_preds = [row[1] for row in knn.predict_proba(x_test)]
roc_auc_score(y_test, knn_preds)

Out[94]: 0.5733444048058391

In [27]:
nn = MLPClassifier(solver="sgd",random_state=0)
nn.hidden_layer_sizes = (60,)
nn.fit(preprocessing.scale(x_train), y_train)
nn_preds = [row[1] for row in nn.predict_proba(preprocessing.scale(x_test))]
roc_auc_score(y_test, nn_preds)

Out[95]: 0.6300754500348547

In [28]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_preds = [row[1] for row in rf.predict_proba(x_test)]
roc_auc_score(y_test, rf_preds)

Out[96]: 0.619440941895272

In [29]:
gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb_preds = [row[1] for row in gnb.predict_proba(x_test)]
roc_auc_score(y_test, gnb_preds)

Out[97]: 0.6316692807643417

In [30]:
#XGBoost seems to be the best. How's it's recall?
recall_score(y_test, xgb.predict(x_test))

Out[98]: 0.18977272727272726

In [31]:
#oof. Let's see the precision-recall curve
#plot_precision_recall_curve(xgb, X_test, y_test)

In [32]:
#What's the max f1 score and threshold?
precision, recall, thresholds = precision_recall_curve(y_test, [r[1] for r in xgb.predict_proba(x_test)])
f1_scores = 2*recall*precision/(recall+precision)
print('Best threshold: ', thresholds[np.argmax(f1_scores)])
print('Best F1-Score: ', np.max(f1_scores))

Best threshold: 0.10472215
Best F1-Score: 0.4484304932735426

In [33]:
#do we at least hack accuracy?
accuracy_score(y_test, xgb.predict(x_test))

Out[101]: 0.7187600904100743

In [34]:
#wait, does that even beat the trivial classifier?
accuracy_score(y_test, [0] * len(y_test))

Out[102]: 0.7158540523086858

In [35]:
#yes I did! But I think the focus should be on the analytical, not predictive capabilities.
#To that end, let's look at some likely readmissions that we could predict to try to tell a story
encounter_df.loc[encounter_df['30_DAY_READMIT'] == 1]
print(encounter_df['ADMIT_ATTR'].unique())

['EMERGENCY' 'ELECTIVE' 'URGENT' 'NEWBORN']

In [36]:
Genit_PEIDs = encounters.loc[encounters['Genitourinary'] > 0].loc[encounters['30_DAY_READMIT'] == 1].PATIENT_ENCOUNTER_ID

In [37]:
#pick a random encounter from the list and take a look at the patient's history
encounter_df.loc[encounter_df.PATIENT_ENCOUNTER_ID == 199510]

,index,PATIENT_ID,PATIENT_ENCOUNTER_ID,ADMIT_TIME,ADMIT_ATTR,DISCHARGE_TIME,DISCHARGE_ATTR,PATIENT_AGE,LOS,MIDPOINT_TIME,ELECTIVE,EMERGENCY,NEWBORN,URGENT,30_DAY_READMIT
58676,141247,76476,199510,2187-12-29T06:56:00.000Z,EMERGENCY,2188-01-01T17:20:00.000Z,SNF,56.868493,3.433333,2187-12-31 00:08:00,0,1,0,0,1


In [38]:
diagnoses_onePEID = diagnoses.loc[diagnoses.PATIENT_ID == 76476].loc[diagnoses.PATIENT_ENCOUNTER_ID == 164373]

In [39]:
encounter_df.loc[encounter_df.PATIENT_ENCOUNTER_ID == 164373]

,index,PATIENT_ID,PATIENT_ENCOUNTER_ID,ADMIT_TIME,ADMIT_ATTR,DISCHARGE_TIME,DISCHARGE_ATTR,PATIENT_AGE,LOS,MIDPOINT_TIME,ELECTIVE,EMERGENCY,NEWBORN,URGENT,30_DAY_READMIT
37925,40226,76476,164373,2188-01-15T13:46:00.000Z,EMERGENCY,2188-01-19T14:55:00.000Z,SNF,56.915068,4.047917,2188-01-17 14:20:30,0,1,0,0,0


In [40]:
PEID = encounters.loc[encounters['Health Status/Services'] > 0].loc[encounters['30_DAY_READMIT'] == 1].PATIENT_ENCOUNTER_ID